Practical Exercise: From Local to Decentralized Computation
Participants should form groups of 3 to 5 and select a simple dataset, such as Iris, Housing Market, or Titanic, 
for the following activities:

Q1: Develop diverse predictive models targeting the selected dataset. 
Each group member should create a distinct model.
💡 Basic flask app
Evaluate the accuracy and performance of your model.
Adapt your model for API access. This API should include a GET predict route that accepts model arguments and 
returns a prediction. Determine a standardized API response format within your group.

In [1]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = sns.load_dataset('titanic')

In [3]:
df.isnull().sum()/len(df)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64

In [4]:
df.drop(columns=['class','adult_male','alive', 'embark_town', 'who', 'deck'], inplace=True)
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,True
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,True
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,True
887,1,1,female,19.0,0,0,30.0000,S,True
888,0,3,female,NaN,1,2,23.4500,S,False
889,1,1,male,26.0,0,0,30.0000,C,True


In [5]:
df.isnull().sum()/len(df)

survived    0.000000
pclass      0.000000
sex         0.000000
age         0.198653
sibsp       0.000000
parch       0.000000
fare        0.000000
embarked    0.002245
alone       0.000000
dtype: float64

In [6]:
df.dropna(inplace=True)

In [7]:
#format data
df['alone'] = df['alone'].apply(lambda x:1 if x else 0)
df['sex'] = df['sex'].apply(lambda x:1 if x=='male' else 0)

In [8]:
df = pd.concat([df.drop('embarked', axis=1),pd.get_dummies(df['embarked'])], axis=1)
df

,survived,pclass,sex,age,sibsp,parch,fare,alone,C,Q,S
0,0,3,1,22.0,1,0,7.2500,0,0,0,1
1,1,1,0,38.0,1,0,71.2833,0,1,0,0
2,1,3,0,26.0,0,0,7.9250,1,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,0,1
4,0,3,1,35.0,0,0,8.0500,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,0,39.0,0,5,29.1250,0,0,1,0
886,0,2,1,27.0,0,0,13.0000,1,0,0,1
887,1,1,0,19.0,0,0,30.0000,1,0,0,1
889,1,1,1,26.0,0,0,30.0000,1,1,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  712 non-null    int64  
 1   pclass    712 non-null    int64  
 2   sex       712 non-null    int64  
 3   age       712 non-null    float64
 4   sibsp     712 non-null    int64  
 5   parch     712 non-null    int64  
 6   fare      712 non-null    float64
 7   alone     712 non-null    int64  
 8   C         712 non-null    uint8  
 9   Q         712 non-null    uint8  
 10  S         712 non-null    uint8  
dtypes: float64(2), int64(6), uint8(3)
memory usage: 52.1 KB


In [23]:
df['age'].min()

0.42

In [10]:
X = df.drop('survived', axis=1)
Y = df['survived']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [12]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
MSE_linear = mean_squared_error(y_test, y_pred)
print(MSE_linear)

0.13980576712136616


In [56]:
test = np.array([3, 1, 39.0, 0, 0, 24.150, 1, 0, 0, 1]).reshape(1,-1)
print(test)

[[ 3.    1.   39.    0.    0.   24.15  1.    0.    0.    1.  ]]


In [57]:
a = model.predict(test)
print(round(a[0]))

0


C:\Users\etien\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [58]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
MSE_linear = mean_squared_error(y_test, y_pred)
print(MSE_linear)

0.2612359550561798


Q2: Generate a consensus prediction by averaging outputs from the group's models, using tools like ngrok for 
inter-computer connectivity. Assess the performance of this aggregated meta-model.

This stage illustrates the creation of a distributed prediction system in a trusted environment. 
The next step involves opening the model to external contributions, which may come from both benign 
and malicious actors.

Introducing Consensus with Slashing Mechanism

Q3: Introduce a weighting system to refine the meta-model's predictions. Weights, ranging from 0 to 1, are 
adjusted with each prediction batch to reflect the accuracy of individual models relative to the group consensus.

Q4: Implement a proof-of-stake consensus mechanism with a slashing protocol. Models must make an initial deposit 
(e.g., 1000 euros) upon registration to participate. This deposit serves as a security measure, ensuring 
participants' commitment to the network's integrity.

Implement penalties (slashing) for actions that undermine network accuracy or trustworthiness. For example, 
consistently inaccurate predictions may result in a loss of deposit. This protocol discourages adverse behaviors 
while encouraging contributions of accurate, reliable predictions. Adjust model weights based on performance and 
penalties, promoting a merit-based system that rewards accuracy and penalizes dishonesty or inaccuracy. Incorporating 
a slashing mechanism enhances the network's reliability and accountability, ensuring that contributions are both 
accurate and made in good faith, thus maintaining the integrity and effectiveness of the decentralized prediction 
system.

To simplify the implementation asume that balance and slashing are done locally. 
Create a Json Database were you track model balance.

In [26]:
from flask import Flask, request

In [88]:
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/predict', methods=['GET'])
def get_predict():
    d = request.args.to_dict()
    l = list(d.values())[:-1]
    for i in range(len(l)):
        if i==2 or i==5:
            l[i] = float(l[i])
        else:
            l[i]=int(l[i])
    if d['port']=='C':
        l.extend([1,0,0])
    elif d['port']=='Q':
        l.extend([0,1,0])
    else:
        l.extend([0,0,1])
    arr = np.array(l).reshape(1,-1)
    
    return f"{model.predict(arr)[0]}"
    """
    survived int64  pclass int64  sex int64  age float64 sibsp int64  parch int64  fare float64 alone int64  C  uint8  Q uint8  S uint8  
    """
app.run(host="0.0.0.0", port=8080)

#pclass sex age sibsp parch fare alone C Q S
#http://localhost:8080/predict?pclass=1&sex=1&age=12.2&sibsp=1&parch=1&fare=10.1&alone=0&port=C
#{'pclass':1, 'sex':1, 'age':12.2, 'sibsp':1, 'parch':1, 'fare':10.1, 'alone':0, 'port':'C Q S'}

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.1.181.28:8080/ (Press CTRL+C to quit)
C:\Users\etien\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [26/Feb/2024 16:47:14] "GET /predict?pclass=1&sex=1&age=12.2&sibsp=1&parch=1&fare=10.1&alone=0&port=C HTTP/1.1" 200 -


In [30]:
a="{'survived': 1, 'pclass': 1, 'sex': 1, 'age': 12.2, 'sibsp': 1, 'parch': 1, 'fare': 10.1, 'alone': 0, 'port': 'C Q S'}"
print(a.replace("'",'"'))

{"survived": 1, "pclass": 1, "sex": 1, "age": 12.2, "sibsp": 1, "parch": 1, "fare": 10.1, "alone": 0, "port": "C Q S"}


In [37]:
for a,b in {"survived": 1, "pclass": 1, "sex": 1, "age": 12.2, "sibsp": 1, "parch": 1, "fare": 10.1, "alone": 0, "port": "C Q S"}.items():
    print(f"{a}={b}&",end="")

survived=1&pclass=1&sex=1&age=12.2&sibsp=1&parch=1&fare=10.1&alone=0&port=C Q S&

In [86]:
    d = {"pclass": 1, "sex": 1, "age": 12.2, "sibsp": 1, "parch": 1, "fare": 10.1, "alone": 0, "port": "C"}
    l = list(d.values())[:-1]
    for i in range(len(l)):
        if i==2 or i==5:
            l[i] = float(l[i])
        else:
            l[i]=int(l[i])
    if d['port']=='C':
        l.extend([1,0,0])
    elif d['port']=='Q':
        l.extend([0,1,0])
    else:
        l.extend([0,0,1])
    arr = np.array(l).reshape(1,-1)
    print(arr)
    print("[[ 3.   1.  39.   0.   0.  24.15 1.   0.   0.   1. ]]")
    print(model.predict(arr))

[[ 1.   1.  12.2  1.   1.  10.1  0.   1.   0.   0. ]]
[[ 3.   1.  39.   0.   0.  24.15 1.   0.   0.   1. ]]
[0.57152357]


C:\Users\etien\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
